In [ ]:
# Load saved models
autoencoder = load_model("autoencoder_model.h5")
scaler = joblib.load("scaler.pkl")

xgb = joblib.load("xgb_model.pkl")
cat = joblib.load("cat_model.pkl")
lgb = joblib.load("lgb_model.pkl")

meta_model = joblib.load("meta_model.pkl")


In [ ]:
# Feature engineering (example, same as before)
X_test_new["log_amount"] = np.log1p(X_test_new["amount"])
X_test_new["transaction_hour"] = X_test_new["step"] % 24
X_test_new["is_night"] = X_test_new["transaction_hour"].apply(lambda x: 1 if 0 <= x <= 6 else 0)
X_test_new["transaction_day"] = X_test_new["step"] // 24
X_test_new["is_weekend"] = X_test_new["transaction_day"] % 7 >= 5

features = ['age', 'gender', 'customer', 'merchant', 'category',
            'log_amount', 'transaction_hour', 'is_night',
            'transaction_day', 'is_weekend']

# Scale
X_test_scaled = scaler.transform(X_test_new[features])

# Autoencoder anomaly score
reconstructed = autoencoder.predict(X_test_scaled)
mse = np.mean(np.power(X_test_scaled - reconstructed, 2), axis=1)
X_test_new["anomaly_score"] = mse

# Prepare for stacked input
X_test_for_stack = X_test_new[features + ["anomaly_score"]]

# Base model predictions
stacked_test = pd.DataFrame({
    "xgb": xgb.predict_proba(X_test_for_stack)[:, 1],
    "cat": cat.predict_proba(X_test_for_stack)[:, 1],
    "lgb": lgb.predict_proba(X_test_for_stack)[:, 1]
})

# Final prediction using stacked meta-model
final_preds = meta_model.predict(stacked_test)
final_probs = meta_model.predict_proba(stacked_test)[:, 1]


In [ ]:
print("Final Prediction Labels:", final_preds)
print("Prediction Probabilities:", final_probs)


In [ ]:
# 📦 1. Import Required Libraries
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# For model prediction
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm

# Optional: suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 🧠 2. Load Saved Models
autoencoder = load_model("autoencoder_model.h5")
scaler = joblib.load("scaler.pkl")

xgb_model = joblib.load("xgb_model.pkl")
cat_model = joblib.load("cat_model.pkl")
lgb_model = joblib.load("lgb_model.pkl")

meta_model = joblib.load("meta_model.pkl")

# 📄 3. Load New Test Data (or use existing)
# Replace this with your actual test DataFrame
X_test_new = pd.read_csv("your_test_file.csv")

# 🛠 4. Feature Engineering (same logic used during training)
X_test_new["log_amount"] = np.log1p(X_test_new["amount"])
X_test_new["transaction_hour"] = X_test_new["step"] % 24
X_test_new["is_night"] = X_test_new["transaction_hour"].apply(lambda x: 1 if 0 <= x <= 6 else 0)
X_test_new["transaction_day"] = X_test_new["step"] // 24
X_test_new["is_weekend"] = X_test_new["transaction_day"] % 7 >= 5

# ✅ Features used for training
features = ['age', 'gender', 'customer', 'merchant', 'category',
            'log_amount', 'transaction_hour', 'is_night',
            'transaction_day', 'is_weekend']

#  5. One-hot encode or convert categorical if necessary (same as training)
# If LabelEncoder or OneHotEncoder was used during training, apply it here too.
# Skipped for simplicity — ensure you apply the same transformations!

#  6. Scale Inputs (same scaler used in training)
X_test_scaled = scaler.transform(X_test_new[features])

# 🔍 7. Compute Autoencoder Anomaly Score
reconstructed = autoencoder.predict(X_test_scaled)
mse = np.mean(np.power(X_test_scaled - reconstructed, 2), axis=1)
X_test_new["anomaly_score"] = mse

# 🧱 8. Prepare Stacked Features (input to base classifiers)
X_test_for_stack = X_test_new[features + ["anomaly_score"]]

# 🤖 9. Base Model Predictions
stacked_test = pd.DataFrame({
    "xgb": xgb_model.predict_proba(X_test_for_stack)[:, 1],
    "cat": cat_model.predict_proba(X_test_for_stack)[:, 1],
    "lgb": lgb_model.predict_proba(X_test_for_stack)[:, 1]
})

# 🧠 10. Meta Model Prediction (Final Output)
final_preds = meta_model.predict(stacked_test)
final_probs = meta_model.predict_proba(stacked_test)[:, 1]

# 📢 11. Output
print("✅ Final Prediction Labels:", final_preds)
print("📊 Final Prediction Probabilities:", final_probs)
